In [1]:
# import libraries
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.integrate import solve_ivp
import plotly.graph_objects as go
from scipy.optimize import minimize_scalar


In [2]:
# load the dataset
df = pd.read_csv('uw_dining_data.csv')

# Clean the data
df.columns = df.columns.str.strip()
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Balance'] = pd.to_numeric(df['Balance'], errors='coerce')

df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_13012\3389981049.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], errors='coerce')


,Date,Time,Charge,Balance,Location
0,2025-07-07,12:21 PM,12.00,1164.60,Local Point
1,2025-07-07,03:26 PM,7.45,1157.15,Starbucks LS - Suzzallo
2,2025-07-07,06:40 PM,15.00,1142.15,Local Point
3,2025-07-08,10:39 AM,14.35,1127.80,Starbucks LS - Suzzallo
4,2025-07-08,05:35 PM,15.00,1112.80,Local Point


In [3]:
# plot the data using Plotly
fig = px.scatter(df, x='Date', y='Balance', title='UW Dining Balance Over Time',
                 labels={'Date': 'Date', 'Balance': 'Balance ($)'})
fig.update_traces(marker=dict(size=8))
fig.show()

In [4]:
# Create time in days since first transaction
df = df.sort_values('Date').reset_index(drop=True)
t0 = df.loc[0, 'Date']
df['t_days'] = (df['Date'] - t0).dt.total_seconds() / (3600 * 24)

In [5]:
# Define parameters for the logistic model
end_date = pd.to_datetime("2025-08-25")  # Set to August 24
B0 = df.loc[0, 'Balance']  # Initial balance
T = (end_date - t0).days   # Duration so that K(t) reaches 0 at Aug 24
T  # Planned total duration of your dining budget in days

49

In [6]:
# Carrying capacity decreases linearly
def K(t):
    return B0 * (1 - t / T)

# Logistic decay with decreasing carrying capacity
def dB_dt(t, B, r):
    return r * B * (1 - B / K(t))

In [7]:
# Ensure t0 and end_date are normalized to midnight
t0 = t0.normalize()
end_date = end_date.normalize()
days_to_aug25 = (end_date - t0).days

# solved model
def solve_logistic(r):
    t_eval = np.linspace(0, days_to_aug25, days_to_aug25 + 1)
    sol = solve_ivp(lambda t, B: dB_dt(t, B, r),
                    [0, days_to_aug25], [B0], t_eval=t_eval)
    return sol

# get mean squared error (MSE)
def mse_loss(r):
    sol = solve_logistic(r)
    # Interpolate model predictions at actual data time points
    interp_model = np.interp(df['t_days'], sol.t, sol.y[0])
    return np.mean((interp_model - df['Balance'])**2)

# find the "best" r and minimize MSE
result = minimize_scalar(mse_loss, bounds=(0.01, 1.0), method='bounded')
best_r = result.x
print(f"Best fit r: {best_r:.4f}")


C:\Users\user\AppData\Local\Temp\ipykernel_13012\827746505.py:7: RuntimeWarning:

divide by zero encountered in divide

c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\integrate\_ivp\rk.py:106: RuntimeWarning:

invalid value encountered in dot

c:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\scipy\integrate\_ivp\rk.py:66: RuntimeWarning:

invalid value encountered in dot



Best fit r: 0.3526


In [8]:
sol = solve_logistic(best_r)
modeled_dates = [t0 + pd.Timedelta(days=float(t)) for t in sol.t]

fig = go.Figure()

# Actual data
fig.add_trace(go.Scatter(x=df['Date'], y=df['Balance'],
                         mode='markers', name='Actual Data',
                         marker=dict(size=8, color='blue')))

# Fitted model
fig.add_trace(go.Scatter(x=modeled_dates, y=sol.y[0],
                         mode='lines', name=f'Fitted Model (r = {best_r:.4f})',
                         line=dict(color='green')))

# Carrying capacity
K_vals = [K(t) for t in sol.t]
fig.add_trace(go.Scatter(x=modeled_dates, y=K_vals,
                         mode='lines', name='Carrying Capacity K(t)',
                         line=dict(dash='dash', color='red')))

fig.update_layout(
    title='UW Dining Balance with Fitted Logistic Decay Model',
    xaxis_title='Date',
    yaxis_title='Balance ($)',
    legend=dict(x=0.99, y=0.99, xanchor='right', yanchor='top'),
    template='plotly_white',
    width=1000,
    xaxis=dict(range=[t0, end_date]),
    yaxis=dict(range=[0, B0])
)
fig.show()


C:\Users\user\AppData\Local\Temp\ipykernel_13012\827746505.py:7: RuntimeWarning:

divide by zero encountered in divide

